In [ ]:
# Import Github repo
from google.colab import files
import zipfile
import os

print('Upload project file')
uploaded = files.upload()

zip = list(uploaded.keys())[0]
with zipfile.ZipFile(zip, 'r') as zip_ref:
  zip_ref.extractall('/content')

print(f'Extracted {zip}')
print('Availale Files/Folders')
for item in os.listdir('/content'):
  print(f'{item}')

Upload project file


Saving NNIK.zip to NNIK.zip
Extracted NNIK.zip
Availale Files/Folders
.config
NNIK.zip
NNIK
sample_data


In [ ]:
# Mount to google drive
from google.colab import drive
import sys
from pathlib import Path

drive.mount('/content/drive')

root = Path('/content/NNIK') if Path('/content/NNIK').exists() else Path('/content/NNIK-main')
print(f'Project root - {root}')
print(f'Project exists - {root.exists()}')

remove = [p for p in sys.path if 'NNIK' in p or 'Scripts' in p]
for path in remove:
  sys.path.remove(path)

project_paths = [
    str(root),
    str(root / 'Scripts'),
    str(root / 'Scripts' / 'Models'),
    str(root / 'Scripts' / 'Models' / 'Machine_Learning'),
    str(root / 'Scripts' / 'Models' / 'Traditional'),
]

for path in project_paths:
  if path not in sys.path:
    sys.path.insert(0, path)

print('Paths configured')

Mounted at /content/drive
Project root - /content/NNIK
Project exists - True
Paths configured


In [ ]:
# Install dependencies
!pip install -q torch torchvision torchaudio scikit-learn pandas matplotlib numpy tqdm pyyaml scipy

In [ ]:
# Import project modules
import importlib.util
import traceback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import yaml
from datetime import datetime

def import_path(module_name, file_path):
  try:
    if not file_path.exists():
      print(f'File not found - {file_path}')
      return None

    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None or spec.loader is None:
      print(f'Couldnt create spec for {module_name}')
      return None

    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module
  except Exception as e:
    print(f'Error importing {module_name} - {str(e)}')
    return None

utils_module = import_path("utils", root / "Scripts" / "utils.py")
data_gen_module = import_path("data_gen", root / "Scripts" / "data_gen.py")
training_module = import_path("training", root / "Scripts" / "training.py")
testing_module = import_path("testing", root / "Scripts" / "testing.py")

if utils_module:
    print("utils.py imported")
if data_gen_module:
    print("data_gen.py imported")
if training_module:
    print("training.py imported")
if testing_module:
    print("testing.py imported")

utils.py imported
data_gen.py imported
training.py imported
testing.py imported


In [ ]:
# Quick test (hope it dont break) -> it works!
data_directory = root / 'data'
print(f'data directory - {data_directory}')
print(f'data directory exists - {data_directory.exists()}')

if data_directory.exists():
  print('\nTraining data - ')
  training_directory = data_directory / 'Training'
  if training_directory.exists():
    training_files = list(training_directory.glob('*.json'))
    print(f'Found {len(training_files)} training files')
    for f in sorted(training_files)[:5]:
      print(f.name)
  print('\nTesting data:')
  testing_directory = data_directory / 'Testing'
  if testing_directory.exists():
    testing_files = list(testing_directory.glob('*.json'))
    print(f'Found {len(testing_files)} testing files')
    for f in sorted(testing_files)[:5]:
      print(f.name)



data directory - /content/NNIK/data
data directory exists - True

Training data - 
Found 16 training files
10_training.json
10_training_solutions.json
3_training.json
3_training_solutions.json
4_training.json

Testing data:
Found 16 testing files
10_testing.json
10_testing_solutions.json
3_testing.json
3_testing_solutions.json
4_testing.json


In [ ]:
from Scripts.training import load_ik_data, train_all_models, evaluate_all_models, create_results_dataframe
from Scripts.testing import create_models
import time
import torch

dof_range = [3, 4, 5, 6]
sample_limit = 1000

trained_models = {}
summary = []

for dof in dof_range:
    print(f"\nDOF={dof} training")

    train_poses = data_directory / 'Training' / f'{dof}_training.json'
    train_solutions = data_directory / 'Training' / f'{dof}_training_solutions.json'

    if not (train_poses.exists() and train_solutions.exists()):
        print(f"no data for DOF={dof}")
        continue

    X_train, y_train = load_ik_data(train_poses, train_solutions)

    if sample_limit and len(X_train) > sample_limit:
        idx = np.random.choice(len(X_train), sample_limit, replace=False)
        X_train, y_train = X_train[idx], y_train[idx]

    print(f"Training data - {X_train.shape} -> {y_train.shape}")

    models_for_dof = create_models(input_dim=X_train.shape[1], output_dim=y_train.shape[1])

    selected_models = {}
    for name in ['ANN', 'KNN', 'ELM', 'RandomForest', 'SVM', 'GPR', 'MDN', 'CVAE']:
        if name in models_for_dof and models_for_dof[name] is not None:
            selected_models[name] = models_for_dof[name]

    print(f"models: {list(selected_models.keys())}")

    trained_models_dof = {}

    for model_name, model in selected_models.items():
        print(f"\nTraining {model_name}")
        print(f"GPU?: {torch.cuda.is_available()}")

        try:
            start_time = time.time()
            model.fit(X_train, y_train)

            training_time = time.time() - start_time

            trained_models_dof[model_name] = {
                'model': model,
                'training_time': training_time,
                'dof': dof
            }

            print(f"{model_name} trained - {training_time:.2f}s")

            summary.append({
                'dof': dof,
                'model': model_name,
                'training_time': training_time,
                'samples': len(X_train),
                'status': 'success'
            })

        except Exception as e:
            print(f"{model_name} failed - {str(e)}")
            summary.append({
                'dof': dof,
                'model': model_name,
                'training_time': 0,
                'samples': len(X_train),
                'status': f'failed: {str(e)[:50]}'
            })

    trained_models[dof] = trained_models_dof

    print(f"\nDOF={dof} training complete - {len(trained_models_dof)} models trained")

print(f"\nTraining Complete")

training_df = pd.DataFrame(summary)
if not training_df.empty:
    print("Training Summary:")
    print(training_df)

    successful = training_df[training_df['status'] == 'success']
    if not successful.empty:
        print(f"\nStatistics:")
        pivot = successful.pivot_table(values='training_time', index='model', columns='dof', aggfunc='mean')
        print(pivot.round(2))

print(f"\nTotal trained models: {sum(len(models) for models in trained_models.values())}")

In [ ]:
from sklearn.metrics import mean_squared_error

all_test_results = []
testing_summary = []

for dof, trained_models_dof in trained_models.items():
    if not trained_models_dof:
        continue

    print(f"\nDOF={dof} testing")

    test_poses = data_directory / 'Testing' / f'{dof}_testing.json'
    test_solutions = data_directory / 'Testing' / f'{dof}_testing_solutions.json'

    if not (test_poses.exists() and test_solutions.exists()):
        print(f"testing data not found for dof={dof}")
        continue

    X_test, y_test = load_ik_data(test_poses, test_solutions)

    if sample_limit and len(X_test) > sample_limit//2:
        idx = np.random.choice(len(X_test), sample_limit//2, replace=False)
        X_test, y_test = X_test[idx], y_test[idx]

    print(f"Testing data: {X_test.shape} -> {y_test.shape}")

    for model_name, model_data in trained_models_dof.items():
        print(f"\nTesting {model_name}")

        try:
            model = model_data['model']
            start_time = time.time()
            y_pred = model.predict(X_test)

            inference_time = time.time() - start_time

            joint_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

            position_rmse = joint_rmse

            test_result = {
                'dof': dof,
                'model': model_name,
                'position_rmse': position_rmse,
                'joint_rmse': joint_rmse,
                'training_time': model_data['training_time'],
                'inference_time': inference_time,
                'inference_time_per_sample': inference_time / len(X_test),
                'test_samples': len(X_test),
                'status': 'success'
            }

            all_test_results.append(test_result)

            print(f"{model_name}: Joint RMSE = {joint_rmse:.4f}")
            print(f"Inference: {inference_time:.3f}s ({inference_time/len(X_test)*1000:.2f}ms/sample)")

            testing_summary.append({
                'dof': dof,
                'model': model_name,
                'joint_rmse': joint_rmse,
                'inference_time': inference_time,
                'status': 'success'
            })

        except Exception as e:
            print(f"{model_name} testing failed: {str(e)}")
            testing_summary.append({
                'dof': dof,
                'model': model_name,
                'joint_rmse': float('inf'),
                'inference_time': 0,
                'status': f'failed: {str(e)[:50]}'
            })

    print(f"\ndof={dof} testing complete")

print(f"\ntesting complete")
print("="*60)

if all_test_results:
    results_df = pd.DataFrame(all_test_results)

    print("Complete Results:")
    display_cols = ['dof', 'model', 'joint_rmse', 'training_time', 'inference_time_per_sample']
    print(results_df[display_cols].round(4))

    results = root / 'results'
    results.mkdir(exist_ok=True)
    results_df.to_csv(results / 'explicit_training_results.csv', index=False)
    print(f"\nResults saved to: {results / 'explicit_training_results.csv'}")

    print(f"\nModel Performance Summary (Average across DOFs):")
    model_summary = results_df.groupby('model').agg({
        'joint_rmse': 'mean',
        'training_time': 'mean',
        'inference_time_per_sample': 'mean'
    }).round(4)
    print(model_summary)

else:
    print("No test results available")